In [7]:
import cv2
import numpy as np
import math
import os
import shutil
from IPython.display import display, clear_output
from numba import jit
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

In [10]:
# @jit()
def get_Hashsample():
    #해시매핑을 위한 기준 이미지입니다. 필요함!!
    sample_img = np.zeros((50,50))
    sample_img = cv2.circle(sample_img, (sample_img.shape[0] //2, sample_img.shape[1] //2), radius=21, color=255, thickness=-1)
    sample_img = cv2.circle(sample_img, (sample_img.shape[0] //2, sample_img.shape[1] //2), radius=14, color=0, thickness=-1)
    return sample_img
sample_img = get_Hashsample()

# @jit()
def find_colorAlignment(img, K, show_one_label=True, blur_count=6, hash_true_condition=0.8):
    recycle_condition = True
    original_image = img.copy()

    blr = cv2.GaussianBlur(original_image, (0, 0), round(blur_count))  #블러처리
    img2 = cv2.cvtColor(blr, cv2.COLOR_BGR2HSV)  #색상변환.

    vectorized = img2.reshape((-1, 3))  #색상rgb 벡터화
    vectorized = np.float32(vectorized)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)  #알고리즘을 멈출조건 (EPS : 0.1=정확도, ITER: 10 = 반복횟수)
    attempts = 7  #시도횟수. 횟수의 결과중 최적의 밀집도가 결과로 나옴
    _, label, _ = cv2.kmeans(vectorized, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)

    sample_hash_resize = sample_img
    sample_hash_avg = sample_hash_resize.mean()
    sample_hash = (sample_hash_resize > sample_hash_avg)

    for i in range(K):
        center1 = np.uint8([[0] * 3 for n in range(K)])  #전체레이블의 중심값을 0,0,0으로 만들어놓음
        selceted_label = label.copy()
        sss = selceted_label.flatten()
        if show_one_label:
            ss = np.where(sss != i, 0, sss)
            center1[i] = [255, 255, 255]  #i 레이블을 255,255,255로 만듬 -> 영역을 contour로 잡기위함
        else:
            ss = sss
        res = center1[ss]
        result_image = res.reshape((original_image.shape))
        # 해쉬매칭
        resize_shape = (50, 50)
        resize_reshape = resize_shape[0] * resize_shape[1]
        hash_true_ratio = resize_reshape * hash_true_condition

        hash_img = result_image[:,:,1]
        img_resize = cv2.resize(hash_img, dsize=resize_shape)
        img_avg = img_resize.mean()
        img_hash = (img_resize > img_avg)

        hash_match = sample_hash == img_hash
        hash_bincount = np.bincount(hash_match.reshape(-1))
        # print("해시조건",hash_true_ratio)
        # print("샘플",hash_bincount[1])
        # print("평균밝기",result_image.mean())
        # 해쉬매칭 조건에 맞을 시 저장, 아닐 시 함수 반복
        if hash_bincount[1] < hash_true_ratio:
            continue
        else:
            # if result_image.mean() > 80.:
            if False:
                # print(result_image.mean())
                continue
            else:
                c_contour, _ = cv2.findContours\
                    (result_image[:,:,1].astype('uint8'), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
                if len(c_contour) > 0:
                    contours_cirles=[]
                    for con in c_contour:
                        area = cv2.contourArea(con)
                        areaRatio = area/(result_image.shape[0]*result_image.shape[1])
                        if 0.4 < areaRatio < 0.6:
                            # print('areaRatio',areaRatio)
                            contours_cirles.append(con)
                    if len(contours_cirles) == 0:
                        continue
                    else:
                        recycle_condition = False
                        x,y,w,h = cv2.boundingRect(contours_cirles[0])
                        x_loc = x+w*0.5
                        y_loc = y+h*0.5
                        error_distance =math.sqrt((original_image.shape[1]/2 - (x+w*0.5))**2 + (original_image.shape[0]/2 - (y+h*0.5))**2)
                        # plt.figure(figsize=(8, 8))
                        # plt.imshow(result_image, cmap='gray')
                        # plt.show()
                        return error_distance, x_loc, y_loc, (w+h)/4

    if recycle_condition:
        # print("cannot find Retry")
        hash_true_condition -= 0.01
        blur_count  *= 1.03
        return find_colorAlignment(img, K, True, blur_count, hash_true_condition)

In [ ]:
# 업체측에서는 center가 0.5mm빗나가면 오류라고 판단.
# 렌즈의 원래 지름은 21mm
# 이미지 크기는 820px
# 820 : 21 = x : 0.5
# x = 19.5       >> 19.5px 이상일 경우 center불량

current_directory = './model_images/DA4649/2_center'
destination_directory = './model_images/DA4649_rotate/train/normal_center'
img_list = os.listdir(current_directory)
total = len(img_list)
labeled = 0
center_count = 0
line_count = 0
normal_count = 0
for i in img_list:
    # st = time.time()
    img = cv2.imread(f'{current_directory}/{i}')
    distance,_,_,_ = find_colorAlignment(img, 7, show_one_label=True, blur_count=6, hash_true_condition=0.7)
    # print(distance, i, time.time() - st)
    if distance >= 10.0 and distance < 19.5: # center불량은 아니지만, line불량일 가능성이 있음.
        shutil.copy(f'{current_directory}/{i}',
                    f'{destination_directory}/center_to_line/{i}')
        line_count += 1
        labeled += 1
    elif distance >= 19.5: # center불량 검출
        shutil.copy(f'{current_directory}/{i}',
                    f'{destination_directory}/center_to_center/{i}')
        center_count += 1
        labeled += 1
    else: # normal
        shutil.copy(f'{current_directory}/{i}',
                    f'{destination_directory}/center_to_normal/{i}')
        normal_count += 1
        labeled += 1


    clear_output(wait=True)
    print(f'{labeled}/{total}')
    print(f'normal : {normal_count}')
    print(f'line   : {line_count}')
    print(f'center : {center_count}')

In [11]:
current_directory = './model_images/DA4649_rotate/train/1_normal'
destination_directory = './model_images/DA4649_rotate/train/normal_center'
img_list = os.listdir(current_directory)
total = len(img_list)
labeled = 0
center_count = 0
line_count = 0
normal_count = 0
for i in img_list:
    st = time.time()
    img = cv2.imread(f'{current_directory}/{i}')
    distance,_,_,_ = find_colorAlignment(img, 7, show_one_label=True, blur_count=6, hash_true_condition=0.7)
    print(time.time()-st)

2.0943965911865234
2.1402769088745117
0.528588056564331
0.5624959468841553
0.49268007278442383
0.46276235580444336
0.4637625217437744
0.5016555786132812


KeyboardInterrupt: 